In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-nlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()

In [2]:
spark

## Read in the Data

In [3]:
reviews = spark.read.parquet("s3://amazon-reviews-pds/parquet")

In [15]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

# Function to convert python object to Java objects
def _to_java_object_rdd(rdd):  
    """ Return a JavaRDD of Object by unpickling
    It will convert each Python object into Java object by Pyrolite, whenever the
    RDD is serialized in batch or not.
    """
    rdd = rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
    return rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)

# Convert DataFrame to an RDD 
JavaObj = _to_java_object_rdd(reviews.rdd)

# Estimate size in bytes
bytes = spark._jvm.org.apache.spark.util.SizeEstimator.estimate(JavaObj)

In [16]:
bytes

95502184

In [18]:
reviews.count()

160796570